In [111]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam,RMSprop,SGD,Adagrad

In [112]:
train_data = pd.read_csv(r'D:\Simplilearn\project\Artificial-Intelligence-Capstone-Project-Datasets-master\Project 3-Retail-Datasets_data\train_data.csv')
train_data.head()

c:\program files\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2,2015-06-30,5735,568,1,1,0,0
1,2,2,2015-06-30,9863,877,1,1,0,0
2,3,2,2015-06-30,13261,1072,1,1,0,1
3,4,2,2015-06-30,13106,1488,1,1,0,0
4,5,2,2015-06-30,6635,645,1,1,0,0


In [113]:
from sklearn.preprocessing import LabelEncoder

In [114]:
from sklearn.model_selection import train_test_split

In [115]:
train_data.drop('Date',axis=1,inplace=True)
train_data.loc[train_data.StateHoliday==0,'StateHoliday'] = '0'
labelencoder= LabelEncoder()
train_data.StateHoliday = labelencoder.fit_transform(train_data['StateHoliday'])
train_data =train_data[~train_data.isin([np.nan, np.inf, -np.inf]).any(1)]

In [116]:
## take 100 shops
train_data = train_data[train_data.Store<=100]
train_data = train_data[train_data.Open == 1]

In [117]:
train_data.reset_index(drop=True, inplace=True)

In [118]:
y = train_data['Sales']
x = train_data.drop(['Sales','Open'],axis=1)

In [119]:
#x.Customers = (x.Customers-x.Customers.mean())/x.Customers.std()

In [120]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
x = std.fit_transform(x)

In [121]:
x

array([[-1.70757208, -0.87819728, -0.48347562,  1.11359979, -0.02637594,
        -0.47415697],
       [-1.67299604, -0.87819728,  0.64160312,  1.11359979, -0.02637594,
        -0.47415697],
       [-1.63842   , -0.87819728,  1.35160427,  1.11359979, -0.02637594,
         2.10900624],
       ...,
       [ 1.68088004, -0.29664439, -1.10245098, -0.89798868, -0.02637594,
         2.10900624],
       [ 1.71545608, -0.29664439, -0.05019287, -0.89798868, -0.02637594,
         2.10900624],
       [ 1.19681545, -0.87819728, -0.29778301, -0.89798868, 25.60857187,
         2.10900624]])

In [122]:
x_train,x_test,y_train,y_test = train_test_split(np.array(x),np.array(y),random_state=42,test_size=0.3)

model_1 = Sequential()
model_1.add(layers.Dense(12, activation='relu', input_shape = (x_train.shape[1],)))
model_1.add(layers.Dense(64, activation='relu'))
model_1.add(layers.BatchNormalization())
model_1.add(layers.Dense(128, activation='relu'))
model_1.add(layers.Dense(64, activation='relu'))
model_1.add(layers.BatchNormalization())
model_1.add(layers.Dense(1))

In [154]:
model_1 = Sequential()
model_1.add(layers.Dense(32, activation='elu', input_shape = (x_train.shape[1],)))
model_1.add(layers.Dense(64, activation='elu'))
model_1.add(layers.Dense(64, activation='elu'))
model_1.add(layers.BatchNormalization())

## block 2
model_1.add(layers.Dense(128, activation='elu'))
model_1.add(layers.Dense(128, activation='elu'))
model_1.add(layers.BatchNormalization())

## block 3
model_1.add(layers.Dense(256, activation='elu'))
model_1.add(layers.Dense(256, activation='elu'))
model_1.add(layers.BatchNormalization())

## block 4
model_1.add(layers.Dense(128, activation='elu'))
model_1.add(layers.Dense(128, activation='elu'))
model_1.add(layers.BatchNormalization())

## block 5
model_1.add(layers.Dense(64, activation='elu'))
model_1.add(layers.Dense(64, activation='elu'))
model_1.add(layers.Dense(32, activation='elu'))
model_1.add(layers.Dense(1))

In [155]:
model_1.compile(loss='mse',
              optimizer = Adam(learning_rate=0.001),
              metrics=['mae'])

In [156]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint('model_3.h5',
                            monitor='loss',
                            mode=min,
                            save_best_only=True,
                            verbose=1)

early_stopping = EarlyStopping(monitor='loss',
                              patience=9,
                              min_delta=0,
                              restore_best_weights=True,
                              verbose=1)

Reduce_ler_rate = ReduceLROnPlateau(monitor='loss',
                                   factor=0.2,
                                   patience=3,
                                   verbose=1,
                                   min_delta=0.001)

callback = [checkpoint,early_stopping,Reduce_ler_rate]

In [157]:
history = model_1.fit(x_train,y_train,epochs=100,batch_size=20,verbose=1,callbacks=callback)

Epoch 1/100
2533/2552 [============================>.] - ETA: 0s - loss: 5013647.5000 - mae: 1547.0806
Epoch 00001: loss improved from inf to 4993100.50000, saving model to model_3.h5
2552/2552 [==============================] - 6s 2ms/step - loss: 4993100.5000 - mae: 1544.2448
Epoch 2/100
2549/2552 [============================>.] - ETA: 0s - loss: 2265282.5000 - mae: 1128.5344
Epoch 00002: loss improved from 4993100.50000 to 2265431.00000, saving model to model_3.h5
2552/2552 [==============================] - 7s 3ms/step - loss: 2265431.0000 - mae: 1128.5953
Epoch 3/100
2549/2552 [============================>.] - ETA: 0s - loss: 2113610.2500 - mae: 1085.0304
Epoch 00003: loss improved from 2265431.00000 to 2113457.25000, saving model to model_3.h5
2552/2552 [==============================] - 6s 2ms/step - loss: 2113457.2500 - mae: 1085.0244
Epoch 4/100
2532/2552 [============================>.] - ETA: 0s - loss: 2009976.8750 - mae: 1062.9116
Epoch 00004: loss improved from 2113457.

Epoch 30/100
2542/2552 [============================>.] - ETA: 0s - loss: 1366395.5000 - mae: 875.2720
Epoch 00030: loss improved from 1387071.25000 to 1368801.37500, saving model to model_3.h5
2552/2552 [==============================] - 10s 4ms/step - loss: 1368801.3750 - mae: 875.5853
Epoch 31/100
2542/2552 [============================>.] - ETA: 0s - loss: 1351835.6250 - mae: 870.0342
Epoch 00031: loss improved from 1368801.37500 to 1352429.12500, saving model to model_3.h5
2552/2552 [==============================] - 9s 4ms/step - loss: 1352429.1250 - mae: 870.1173
Epoch 32/100
2545/2552 [============================>.] - ETA: 0s - loss: 1331265.5000 - mae: 864.1417
Epoch 00032: loss improved from 1352429.12500 to 1330876.12500, saving model to model_3.h5
2552/2552 [==============================] - 9s 4ms/step - loss: 1330876.1250 - mae: 864.0894
Epoch 33/100
2541/2552 [============================>.] - ETA: 0s - loss: 1307981.6250 - mae: 856.2114
Epoch 00033: loss improved from 

2552/2552 [==============================] - 13s 5ms/step - loss: 1053798.3750 - mae: 748.8514
Epoch 59/100
2540/2552 [============================>.] - ETA: 0s - loss: 1054185.7500 - mae: 750.2156
Epoch 00059: loss did not improve from 1053798.37500
2552/2552 [==============================] - 10s 4ms/step - loss: 1054668.7500 - mae: 750.4318
Epoch 60/100
2549/2552 [============================>.] - ETA: 0s - loss: 1047672.5000 - mae: 745.2038
Epoch 00060: loss improved from 1053798.37500 to 1047679.25000, saving model to model_3.h5
2552/2552 [==============================] - 10s 4ms/step - loss: 1047679.2500 - mae: 745.2357
Epoch 61/100
2547/2552 [============================>.] - ETA: 0s - loss: 1041059.7500 - mae: 742.6868
Epoch 00061: loss improved from 1047679.25000 to 1040776.68750, saving model to model_3.h5
2552/2552 [==============================] - 10s 4ms/step - loss: 1040776.6875 - mae: 742.7044
Epoch 62/100
2547/2552 [============================>.] - ETA: 0s - loss: 10

Epoch 88/100
2547/2552 [============================>.] - ETA: 0s - loss: 849647.5000 - mae: 668.4875
Epoch 00088: loss improved from 857981.43750 to 850165.62500, saving model to model_3.h5
2552/2552 [==============================] - 10s 4ms/step - loss: 850165.6250 - mae: 668.6979
Epoch 89/100
2542/2552 [============================>.] - ETA: 0s - loss: 845742.5625 - mae: 666.8588
Epoch 00089: loss improved from 850165.62500 to 846068.06250, saving model to model_3.h5
2552/2552 [==============================] - 11s 4ms/step - loss: 846068.0625 - mae: 666.9547
Epoch 90/100
2552/2552 [==============================] - ETA: 0s - loss: 837413.7500 - mae: 662.7102
Epoch 00090: loss improved from 846068.06250 to 837413.75000, saving model to model_3.h5
2552/2552 [==============================] - 10s 4ms/step - loss: 837413.7500 - mae: 662.7102
Epoch 91/100
2547/2552 [============================>.] - ETA: 0s - loss: 830736.6875 - mae: 661.2625
Epoch 00091: loss improved from 837413.7500

In [158]:
from keras.models import load_model
model_1 = load_model('model_3.h5')

In [159]:
y_pred = model_1.predict(x_test)

In [160]:
import math

In [161]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [162]:
math.sqrt(mean_squared_error(y_test,y_pred))

874.4867336424072